<a href="https://colab.research.google.com/github/IMIYOU/DeepFaceLab/blob/master/DeepFaceLab_CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

无需多说，选择好你的模式，一路运行即可！

此脚本仅用于学习，请勿滥用。如有疑问，联系QQ：269777955 ！

In [0]:
#@title 查看分配到的设备
# 查看设备，是K80还是T4,如果是K80...！
! /opt/bin/nvidia-smi

# 准备工作！！！！

第一次运行 安装程序并准备好workspace目录或文件
这里你可以选着两种方式
1. 使用默认的workspace，你无需自己上传，仅用于熟悉操作。
2. 通过Google Drive （谷歌云盘）上传自己的workspace到云盘DeepFaceLab目录下。



谷歌云盘地址：https://drive.google.com/drive/my-drive  
注意：谷歌网址现在都需要“科学上网”才能访问。  

挂载时会提示你点击链接，获取云盘授权码，填入后按回车键！

In [0]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


创建程序所的文件夹！！！

In [0]:
#@title 创建云盘文件目录，并且进入DeepFaceLab目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/
!mkdir workspace
%cd /content/drive/My Drive/DeepFaceLab/workspace/
!mkdir model
%cd /content/drive/My Drive/DeepFaceLab/

使用默认的workspace，你无需自己上传，仅用于熟悉操作。
如果你需要上传自己的素材请跳过此步骤！

In [0]:
#@title 加载示例项目workspace 
# 1. 练手可以使用这一行，直接git clone一个workspace
%cd /content/drive/My Drive/DeepFaceLab/
!rm -rf workspace
!git clone https://github.com/dream80/DFLWorkspace.git workspace

解压自己的项目文件，建立COLAB缓存提高读取速度！！！

In [0]:
#@title 解压自己的项目workspace
#需要先将workspace.zip 上传到谷歌云盘的DeepFaceLab目录下面
%cd /content/drive/My Drive/DeepFaceLab/
!unzip  workspace.zip

安装最新版本的DeepFaceLab程序，如无更新无需重复操作！如QQ群有提示大家更新，先到云盘DeepFaceLab目录下删除DeepFaceLab_CN文件夹再进行操作！

In [0]:
#@title 开始安装

print("最新版加载中....")
!git clone https://github.com/IMIYOU/DeepFaceLab_CN.git
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_CN
!pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0


# 模式一 需要在COLAB上提取训练转换的从这里开始

第一步 挂载成功并进入DeepFaceLab目录后，继续下列操作！！！

In [0]:
#@title 开始挂载 
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#创建DeepFaceLab目录，并且进入目录
%cd /content/drive/My Drive/
!mkdir DeepFaceLab
%cd /content/drive/My Drive/DeepFaceLab/


第二步 安装程序运行所需的Python依赖环境！！！


In [0]:
#@title 安装Python依赖

%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_CN
!pip install -r requirements-colab.txt

!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0


第三步 提取人脸素材！

提取人脸这个步骤其实是分两次的。第一次转src，第二次选dst，很多对Deepfacelab不了解的人可能只点了一个，后面就会报错了。

注！！！SRC视频转图片时可自行选择每秒帧数，DST视频转图片时不要输入每秒转换帧数直接回车，避免出现DST最后合成视频不完整！

In [0]:
#@title 开始提取
target = "dst" #@param ["src","dst"]
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_CN
if target=="src":
  !python main.py videoed extract-video --input-file ../workspace/data_src.mp4 --output-dir ../workspace/data_src/
  !python main.py extract --input-dir ../workspace/data_src --output-dir ../workspace/data_src/aligned --detector s3fd --debug-dir ../workspace/data_src/aligned_debug
else:
  !python main.py videoed extract-video --input-file ../workspace/data_dst.mp4 --output-dir ../workspace/data_dst/
  !python main.py extract --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/aligned --detector s3fd --debug-dir ../workspace/data_dst/aligned_debug
  


排序工具，方便你在云盘中对素材进行检查和筛选！！！！

In [0]:
#@title 开始排序
target = "src" #@param ["src","dst"]
order = "hist" #@param ["blur", "face", "face-dissim", "face-yaw", "face-pitch", "hist", "hist-dissim", "brightness", "hue", "black", "origname", "oneface", "final", "final-no-blur"]

if target=="src":
  #Src排序，可以通过谷歌云盘查看结果，删除不良图片
  cmd = "main.py sort --input-dir ../workspace/data_src/aligned --by "+order
  
else:
  cmd = "main.py sort --input-dir ../workspace/data_dst/aligned --by "+order

!python $cmd

第四步 模型训练
*  支持H128,SAE,DF, LIAEF128等模型，根据自己的情况选择模型。
*  第一次需要配置模型参数，不懂的直接回车默认。
*  不想训练了可以点击停止，停止时会抛出异常，但是没什么关系。下次可以继续训练

In [0]:
#@title 开始训练
Model = "SAEHD" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR","Quick96","IMU","IMUHD"]

%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_CN/"
cmd = "main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model "+Model+" --no-preview"
!python $cmd

**模型预览方法请参考：**

简单描述：左侧->文件->/driver/..../model/ *preview*.jpg->双击  
  
这种方法比在脚本中预览和在网盘预览都要方便，并且无需启动预览配置！

第五步. 转换输出

训练模型的时候选了什么，这里就选什么  
比如训练H128,这里就选H128


In [0]:
#@title 开始转换图片

Model = "H128" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR", "Quick96","IMU","IMUHD"]
%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_CN/"
cmd = " main.py convert --input-dir ../workspace/data_dst --output-dir ../workspace/data_dst/merged --aligned-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model
!python $cmd

In [0]:
#@title 开始图片转视频
%cd "/content/drive/My Drive/DeepFaceLab/DeepFaceLab_CN/"
!python main.py videoed video-from-sequence --input-dir ../workspace/data_dst/merged --output-file ../workspace/result.mp4 --reference-file ../workspace/data_dst.mp4

# 模式二 只需要在COLAB上进行模型训练的从这里开始
当你第二次开始训练，或者掉线之后继续训练时并不需要执行上面所有的步骤。只需要下面简单的几个步骤。
1. 挂载云盘
2. 安装依赖
3. 开始训练  
**注意：**workspace.zip只包含src和dst，不包含model,否则模型会被覆盖。！！！

In [0]:
#@title 一键运行
#挂载谷歌云盘
#点击链接授权，复制授权码，填入框框，然后回车。

Model = "SAE" #@param ["SAEHD", "SAE", "H128", "LIAEF128", "DF", "AVATAR" , "Quick96","IMU","IMUHD"]


from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#重新覆盖解压，提升加载速度，可节省很多时间
%cd /content/drive/My Drive/DeepFaceLab/
!unzip  -o workspace.zip


# 进入DeepFaceLab_CN目录
%cd /content/drive/My Drive/DeepFaceLab/DeepFaceLab_CN

# 安装Python依赖
!pip install -r requirements-colab.txt
!pip install --upgrade scikit-image
!sudo apt-get install cuda-10-0

# 开始训练SAE ，如果是其他模型，修改后面的参数即可，比如H128。
cmd ="main.py train --training-data-src-dir ../workspace/data_src/aligned --training-data-dst-dir ../workspace/data_dst/aligned --model-dir ../workspace/model --model " + Model +" --no-preview"
!python $cmd 

使用脚本练模型的时候常常会需要上传，解压，压缩，下载等，下面为大家提供一个解压和压缩的脚本  
所有压缩的文件都存在zip目录，名字都叫data.zip，压缩前选一下压缩那个文件夹下的内容。  
解压的时候也是上传一个data.zip，选择解压到哪里就好了  




# 工具


In [0]:
#@title 删除各目录图片（慎用！删除图片文件！）

Mode = "云盘根目录" #@param ["云盘根目录", "DeepFaceLab", "workspace", "SRC视频转换图片", "DST视频转换图片", "SRC人脸遮罩图片", "DST人脸遮罩图片", "转换合成图片", "SRC人脸素材图片", "DST人脸素材图片"]
if Mode == "云盘根目录":
  %cd "/content/drive/My Drive"
  cmd = "rm *.jpg"  
!$cmd
if Mode == "DeepFaceLab":
  %cd "/content/drive/My Drive/DeepFaceLab"
  cmd = "rm *.jpg"  
!$cmd
if Mode == "workspace":
  %cd "/content/drive/My Drive/DeepFaceLab/workspace"
  cmd = "rm *.jpg"  
!$cmd
if Mode == "SRC视频转换图片":
  %cd "/content/drive/My Drive/DeepFaceLab/workspace/data_src"
  cmd = "rm *.jpg"  
!$cmd
if Mode == "DST视频转换图片":
  %cd "/content/drive/My Drive/DeepFaceLab/workspace/data_dst"
  cmd = "rm *.jpg"  
!$cmd
if Mode == "SRC人脸遮罩图片":
  %cd "/content/drive/My Drive/DeepFaceLab/workspace/data_src/aligned_debug"
  cmd = "rm *.jpg"  
!$cmd
if Mode == "DST人脸遮罩图片":
  %cd "/content/drive/My Drive/DeepFaceLab/workspace/data_dst/aligned_debug"
  cmd = "rm *.jpg"  
!$cmd
if Mode == "SRC人脸素材图片":
  %cd "/content/drive/My Drive/DeepFaceLab/workspace/data_src/aligned"
  cmd = "rm *.*"  
!$cmd
if Mode == "DST人脸素材图片":
  %cd "/content/drive/My Drive/DeepFaceLab/workspace/data_dst/aligned"
  cmd = "rm *.*"  
!$cmd
if Mode == "转换合成图片":
  %cd "/content/drive/My Drive/DeepFaceLab/workspace/data_dst/merged"
  cmd = "rm *.jpg"  
!$cmd
print("Done!")
